# Merkle Trees in Python
This Python notebook serves as a reference in understanding how Merkle Trees work, their functionalities, and simulates how the Merkle tree data structure works typically for blockchains.


## What Are Merkle Trees?
Merkle trees are hash-based binary tree data structures used to efficienctly verify the integrity and consistency of large datasets. Merkle Trees are used in blockchain systems like Bitcoin and Ethereum to summarize and secure data, such as transactions in a block.

1. Key Properties 

- Tamper-Evidence: Merkle trees ensure data integrity by detecting tampering. A change in a data item (e.g., a transaction) alters its leaf hash, causing changes to propagate up and eventually change the Merkle root. A mismatch between the computed and expected root signals that tampering has occurred.
- Efficiency: Merkle trees allow quick verification of data within a large dataset by only requiring minimal information. This is done by utilising Merkle proofs to prove that a specific data block is part of a larger dataset, without needing to send the entire dataset, requiring only O(log n) hashes to prove a data item's inclusion. 
- Scalability: By summarizing thousands of data items into a single Merkle root, Merkle trees enable compact storage (e.g., in blockchain headers) and efficient processing, making it ideal for large datasets and distributed systems.

2. Structure 

- At its core, a merkle tree is a balanced binary tree that utilises cryptographic hashes. Similar to a binary tree a merkle tree has:
    - Leaf Nodes which store cryptographic hashes of data items (typically, SHA-256 is used for hashing). Each node has at most 2 children.
    - Parent / Branch Nodes that store hashes of their two children's hashes, of which the hashes are computed recursively up to a single Merkle Root (i.e. Root Node). 
    - A Balanced Hierarchical Structure where the tree has a height of O(log n) for n leaves, with leaf nodes at the bottom and a single root at the top, ensuring efficient operations.

3. Merkle Proofs

- Merkle proofs are a key feature of Merkle trees, enabling efficient and secure verification of data inclusion. A Merkle proof is a sequence of sibling hashes along the path from a leaf to the root, used to verify that a specific data item is part of the tree without needing the entire dataset.
- How it Works: To prove a leaf’s inclusion, provide its hash and the sibling hashes at each level. The verifier recomputes the Merkle root by hashing the leaf with its sibling, then hashing the result with the next sibling, up to the root. If the recomputed root matches the known root, the proof is valid.
- Efficiency: While the input size grows linearly O(n), Merkle proofs require only O(log n) hashes due to the tree’s logarithmic height, making verification fast even for large datasets.
- Lightweight: Proofs enable light clients (e.g., Bitcoin’s SPV clients / Crypto Wallets) to verify transactions without storing the full blockchain, ideal for resource-constrained devices.
- Trustless: Proofs rely on cryptographic hashes, allowing verification without trusting the data provider, as long as the Merkle root is trusted.
- Tamper-Evidence: An incorrect proof results in a mismatched root, ensuring data integrity.

## Components of a Merkle Tree

To implement the simluation of a merkle tree data structure in Python, this notebook comprises of the following components to build the merkle tree:

1. Hash Function
2. Leaf Node Creation
3. Tree Construction
4. Proof Generation
5. Proof Verification

### 1. Hash Function
First lets start with the hash function. The hash function converts data into a fixed-size SHA-256 hash, a key step for Merkle trees in blockchains like Bitcoin and Ethereum. We use Python’s `hashlib` module to hash a string input into a 32-byte hash, ensuring data integrity.  Notice how the hash always outputs 32 bytes or 64 characters when displayed as a hexadecimal (2 chars per byte). Run the cell below to see the output.

In [ ]:
import hashlib
def sha256_hash(data: str) -> bytes:
    """Hash a string using SHA-256, returning a 32-byte hash.
    Parameters:
        data: Input string to hash (encoded as UTF-8).
    Returns:
        32-byte SHA-256 hash.
    """
    return hashlib.sha256(data.encode('utf-8')).digest()

input_data1 = "transaction1"
input_data2 = "txn1"
output_str1 =  sha256_hash(input_data1)
output_str2 =  sha256_hash(input_data2)
print(f"output_str1 (bytes): {output_str1}")
print(f"output_str2 (bytes): {output_str2}")
print(f"output_str1 (hex): {output_str1.hex()}")
print(f"output_str2 (hex): {output_str2.hex()}")

output_str1 (bytes): b'\xbd\xe4i>U\xa36\xff\x81\xab#\x8c\xe2\x0c\xae\x1d\xd9\xc8\xba\x03\xb9\xb8\xf49c\xf5V\x9b\xf3\xcfR)'
output_str2 (bytes): b'\xbc\xad\x10\xeeS\xaf1\xca\xc1.5\x8e\xd6\xb8F\xd5\x0e~\xc4\xb5f\x9e\xf6\xae\xd5\xb5s\x8f\xe6$\x8b\x16'
output_str1 (hex): bde4693e55a336ff81ab238ce20cae1dd9c8ba03b9b8f43963f5569bf3cf5229
output_str2 (hex): bcad10ee53af31cac12e358ed6b846d50e7ec4b5669ef6aed5b5738fe6248b16


### 2. Leaf Node Creation

Leaf nodes form the base of a Merkle tree by hashing each data item. This function basically takes a list of strings (e.g., transaction IDs) and returns their SHA-256 hashes, creating the tree’s bottom layer.

In [ ]:
def create_leaf_nodes(data_items: list[str]) -> list[bytes]:
    """Create leaf nodes by hashing a list of strings.
    Parameters:
        data_items: List of strings (e.g., transaction IDs).
    Returns:
        List of 32-byte hashes (leaf nodes).
    """
    return [sha256_hash(item) for item in data_items]

transactions = ["txn1", "txn2", "txn3"]
leaf_nodes = create_leaf_nodes(transactions)
for i in range(len(leaf_nodes)):
    print(f"Leaf {i+1}: {leaf_nodes[i].hex()}")

Leaf 1: bcad10ee53af31cac12e358ed6b846d50e7ec4b5669ef6aed5b5738fe6248b16
Leaf 2: fb4bb01813bcb9f6ac5f585504a29822eaf13cb014c84cb55d05e4ea000b7a10
Leaf 3: 093e863cfe257b8fb8f72150d66e04cd4c5e1e3bee582ebbd562083b733555a1


### 3. Tree Construction
After hashing our transactions and simulating the creation of leaf nodes, we now have to build our merkle tree. We do this by pairing leaf nodes and hashing them to form parent nodes, repeating this until we reach a single Merkle root. If a level has an odd number of nodes, we duplicate the last node to maintain a binary structure. We store the tree as a list of levels for clarity.

Note: 
- A tree is balanced when the number of leaves n is a power of 2 (e.g. 2,4,8,16)
- When we have an odd number of leaves (e.g. 5), duplication of the last node ensures we have a tree height of <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mo stretchy="false">⌈</mo><msub><mrow><mi>log</mi><mo>⁡</mo></mrow><mn>2</mn></msub><mo stretchy="false">(</mo><mi>n</mi><mo stretchy="false">)</mo><mo stretchy="false">⌉</mo></mrow><annotation encoding="application/x-tex"> \lceil \log_2(n) \rceil </annotation></semantics></math> = ceiling of the base 2 log of n
- so if we have 5 leaf nodes: 
<math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><msub><mrow><mi>log</mi><mo>⁡</mo></mrow><mn>2</mn></msub><mo stretchy="false">(</mo><mn>5</mn><mo stretchy="false">)</mo><mo>≈</mo><mn>2.322</mn></mrow><annotation encoding="application/x-tex"> \log_2(5) \approx 2.322 </annotation></semantics></math>. The ceiling of that <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mo stretchy="false">⌈</mo><mn>2.322</mn><mo stretchy="false">⌉</mo><mo>=</mo><mn>3</mn></mrow><annotation encoding="application/x-tex"> \lceil 2.322 \rceil = 3 </annotation></semantics></math>

The code below utilises the dataclass decorator to automatically initialise the Node class when it is used and Optional is used to represent that a node may have a parent, left and right child or none at all. All nodes must have a hash value. 

Definitions (in general):
- Root Node: No Parent, has Left and Right Child
- Parent / Branch / Non-leaf Node: Has Parent, Left and Right Child
- Leaf Node: Has Parent, no Left and no Right Child

In [31]:
import hashlib
from dataclasses import dataclass
from typing import Optional

@dataclass
class Node:
    """Represents a node in the Merkle tree."""
    # all nodes must have a hash, but it is optional for it to have parents and children. 
    hash: bytes
    parent: Optional['Node'] = None
    left_child: Optional['Node'] = None
    right_child: Optional['Node'] = None

def build_merkle_tree(leaves: list[bytes]) -> tuple[Optional[Node], list[list[Node]]]:
    """Construct a Merkle tree from a list of leaf nodes.
    Parameters:
        leaves: List of 32-byte hashes (leaf nodes).
    Returns:
        tuple of (root node, list of nodes representing the entire merkle tree)
    """
    if not leaves:
        return None, []
    
    # create list of leaf nodes
    leaf_nodes = [Node(hash=leaf) for leaf in leaves]
    current_level = leaf_nodes
    merkle_tree = []

    while len(current_level) > 1:
        next_level = []
        # for tracking all nodes including duplicates
        current_level_nodes = []
        for i in range(0, len(current_level), 2):
            left_node = current_level[i]
            # duplicate left node if no sibling 
            right_node = current_level[i+1] if (i+1 < len(current_level)) else Node(hash=left_node.hash)
            # current_level_nodes.extend([left_node.hash.hex()[:4], right_node.hash.hex()[:4]])
            current_level_nodes.extend([left_node, right_node])
            # create parent node 
            parent_hash = sha256_hash(left_node.hash + right_node.hash)
            parent_node = Node(hash=parent_hash, left_child=left_node, right_child=right_node)
            # point children to their parent 
            left_node.parent = parent_node
            right_node.parent = parent_node
            next_level.append(parent_node)
        merkle_tree.append(current_level_nodes)
        current_level = next_level
    # identify the root and add it to the tree
    merkle_root = current_level[0]
    merkle_tree.append(current_level)
    return merkle_root, merkle_tree

def display_merkle_tree(merkle_root: Optional[Node], merkle_tree: list[list[Node]]) -> None:
    """helper function to print the Merkle tree structure starting from the root."""
    if not merkle_root:
        print("No trees here fam.")
        return None
    # get no. of levels in the tree 
    num_levels = len(merkle_tree)
    print(f"Merkle Root: {merkle_root.hash.hex()[:4]}")
    print(f"Height = number of edges in tree: {num_levels-1}")
    # traverse from root to leaves
    for i in range(num_levels -1, -1, -1):
        temp = []
        for node in merkle_tree[i]:
            temp.append(node.hash.hex()[:4])
        print(f"Level {i}: {temp}")


def sha256_hash(data: str | bytes) -> bytes:
    """Hash string or bytes using SHA-256, returning a 32-byte hash."""
    if isinstance(data, str):
        data = data.encode('utf-8')
    return hashlib.sha256(data).digest()

def create_leaf_nodes(data_items: list[str]) -> list[bytes]:
    """Create leaf nodes by hashing a list of strings, returning a list of 32-byte hashes."""
    return [sha256_hash(item) for item in data_items]

# build the tree
transactions = ["txn1", "txn2", "txn3", "txn4", "txn5"]
leaf_nodes = create_leaf_nodes(transactions)
merkle_root, merkle_tree = build_merkle_tree(leaf_nodes)
display_merkle_tree(merkle_root=merkle_root, merkle_tree=merkle_tree)

Merkle Root: 5537
Height = number of edges in tree: 3
Level 3: ['5537']
Level 2: ['8ce0', 'fee4']
Level 1: ['6883', '58e7', '06f3', '06f3']
Level 0: ['bcad', 'fb4b', '093e', '5517', 'e8a2', 'e8a2']


With 5 transactions, this implementation duplicates the node at each level since the node remains unpaired at subsequent levels.
```
Level 3:                                     H12345555 (5537)
                                             /          \
Level 2:                          H1234 (8ce0)       H5555 (fee4)
                                  /      \           /      \
Level 1:                H12 (6883)   H34 (58e7)  H55 (06f3) H55 (06f3)
                        /    \       /    \      /         /   \
Level 0:   H1 (bcad) H2 (fb4b) H3 (093e) H4 (5517) H5 (e8a2) H5 (e8a2)
```

### 4. Proof Generation
With the construction of our Merkle tree, we can now implement the Merkle proof, which provides the minimal set of sibling hashes needed to reconstruct the path from a leaf node to the root hash. The merkle proof generated then allows a verifier to recompute the root hash and confirm the transaction’s inclusion. We hash the input transaction, find its first occurrence in the leaf level, and generate the proof by collecting sibling hashes up to the level before the root.

Lets treat our data ["txn1", "txn2", "txn3", "txn4", "txn5"] to be H1 - H5 when hashed. This means hash('txn1') = H1, with the parent of H1 and H2 being H12 and so on. 

From the previous step, our tree structure is:
```
Level 3:         H12345555 (root)
                /         \
Level 2:   H1234        H5555 
           /    \        /  \
Level 1: H12    H34    H55  H55 (dupe)
         / \    / \    /  \
Level 0: H1 H2 H3 H4  H5  H5(dupe)
```
Assuming we want to generate a proof starting from H2, which is right the child of H12, we collect H2's sibling, H1 as we traverse upwards. This means that at level 0, we start building our proof with:

[(H1, left)]

Next, we move to H2's parent which is H12 at level 1. Repeating the same logic, we find the sibling of H12, which is H34. Now our proof should look something like this:

[(H1, left), (H34, right)]

And now we repeat the same process reaching the root. 

[(H1, left), (H34, right), (H5555, right)]

This proof now allows a verifier to recompute the root hash starting from H2, of which
hash(H1 + H2) → H12, hash(H12 + H34) → H1234, hash(H1234 + H5555) → H12345555. 

Notice how we don't have to start traversal from H3 H4 and build it up to H34, as we already have the previous hash of the concatenation of bytes at H34 looking at the other child of the parent node H1234. This continues with calculating the hash for H1234 (left) and H5555 (right), eventually reaching the merkle root hash of H12345555, of which a match in the merkle root hash confirms that hash('txn2') exists in the tree. 

In [ ]:
import hashlib
from dataclasses import dataclass
from typing import Optional

def generate_merkle_proof(merkle_tree: list[list[Node]], transaction: str) -> list[tuple[bytes, str]]:
    """Generate a Merkle proof for a transaction found in the merkle tree
    Parameters:
        merkle_tree: list of nodes at each level in the merkle tree
        transaction: the transaction string to generate the proof e.g. 'txn5'
    Returns:
        A proof represented as a list of tuples(sibling_hash, position)
    """
    if not merkle_tree: 
        print("Error: not a valid Merkle tree.")
        return []
    
    leaf_hash = sha256_hash(transaction)
    leaf_level = merkle_tree[0]

    # find a matching leaf hash
    leaf_index = None
    for i, node in enumerate(leaf_level):
        if node.hash == leaf_hash:
            leaf_index = i
            break
    if leaf_index is None:
        print("Error: Transaction not found.")
        return []
    
    # generate the proof
    proof = []
    current_index = leaf_index
    num_levels = len(merkle_tree)

    # traverse the tree to find sibling hashes
    for level in range(num_levels-1):
        if current_index % 2 == 0: # handle even (left child)
            sibling_index = current_index + 1
            position = 'right'
        else: # handle odd (right child)
            sibling_index = current_index - 1
            position = 'left'
        sibling_hash = merkle_tree[level][sibling_index].hash
        proof.append((sibling_hash, position))
        # floor to get the parent index 
        current_index = current_index // 2
    return proof

def display_merkle_proof(merkle_proof: list[tuple[bytes, str]]) -> None:
    """helper function to display the Merkle proof generated"""
    if not merkle_proof:
        print("Error: merkle proof provided is invalid")
        return None
    print("Generated Merkle Proof: ")
    for i, (sibling_hash, position) in enumerate(merkle_proof):
        print(f"Level {i}: Sibling Hash: {sibling_hash.hex()[:4]} ({position})")

@dataclass
class Node:
    """Represents a node in the Merkle tree."""
    # all nodes must have a hash, but it is optional for it to have parents and children. 
    hash: bytes
    parent: Optional['Node'] = None
    left_child: Optional['Node'] = None
    right_child: Optional['Node'] = None

def build_merkle_tree(leaves: list[bytes]) -> tuple[Optional[Node], list[list[Node]]]:
    """Construct a Merkle tree from a list of leaf nodes.
    Parameters:
        leaves: List of 32-byte hashes (leaf nodes).
    Returns:
        tuple of (root node, list of nodes representing the entire merkle tree)
    """
    if not leaves:
        return None, []
    
    # create list of leaf nodes
    leaf_nodes = [Node(hash=leaf) for leaf in leaves]
    current_level = leaf_nodes
    merkle_tree = []

    while len(current_level) > 1:
        next_level = []
        # for tracking all nodes including duplicates
        current_level_nodes = []
        for i in range(0, len(current_level), 2):
            left_node = current_level[i]
            # duplicate left node if no sibling 
            right_node = current_level[i+1] if (i+1 < len(current_level)) else Node(hash=left_node.hash)
            # current_level_nodes.extend([left_node.hash.hex()[:4], right_node.hash.hex()[:4]])
            current_level_nodes.extend([left_node, right_node])
            # create parent node 
            parent_hash = sha256_hash(left_node.hash + right_node.hash)
            parent_node = Node(hash=parent_hash, left_child=left_node, right_child=right_node)
            # point children to their parent 
            left_node.parent = parent_node
            right_node.parent = parent_node
            next_level.append(parent_node)
        merkle_tree.append(current_level_nodes)
        current_level = next_level
    # identify the root and add it to the tree
    merkle_root = current_level[0]
    merkle_tree.append(current_level)
    return merkle_root, merkle_tree

def display_merkle_tree(merkle_root: Optional[Node], merkle_tree: list[list[Node]]) -> None:
    """helper function to print the Merkle tree structure starting from the root."""
    if not merkle_root:
        print("merkle tree provided is invalid")
        return None
    # get no. of levels in the tree 
    num_levels = len(merkle_tree)
    print(f"Merkle Root: {merkle_root.hash.hex()[:4]}")
    print(f"Height = number of edges in tree: {num_levels-1}")
    # traverse from root to leaves
    for i in range(num_levels -1, -1, -1):
        current_nodes = []
        for node in merkle_tree[i]:
            current_nodes.append(node.hash.hex()[:4])
        print(f"Level {i}: {current_nodes}")

def sha256_hash(data: str | bytes) -> bytes:
    """Hash string or bytes using SHA-256, returning a 32-byte hash."""
    if isinstance(data, str):
        data = data.encode('utf-8')
    return hashlib.sha256(data).digest()

def create_leaf_nodes(data_items: list[str]) -> list[bytes]:
    """Create leaf nodes by hashing a list of strings, returning a list of 32-byte hashes."""
    return [sha256_hash(item) for item in data_items]

transactions = ["txn1", "txn2", "txn3", "txn4", "txn5"]
leaf_nodes = create_leaf_nodes(transactions)
merkle_root, merkle_tree = build_merkle_tree(leaf_nodes)
display_merkle_tree(merkle_root=merkle_root, merkle_tree=merkle_tree)
merkle_proof = generate_merkle_proof(merkle_tree=merkle_tree, transaction='txn1')
display_merkle_proof(merkle_proof=merkle_proof)

Merkle Root: 5537
Height = number of edges in tree: 3
Level 3: ['5537']
Level 2: ['8ce0', 'fee4']
Level 1: ['6883', '58e7', '06f3', '06f3']
Level 0: ['bcad', 'fb4b', '093e', '5517', 'e8a2', 'e8a2']
Generated Merkle Proof: 
Level 0: Sibling Hash: fb4b (right)
Level 1: Sibling Hash: 58e7 (right)
Level 2: Sibling Hash: fee4 (right)


### 5. Proof Verification
Now we have the last step which is proof verification. 

### Putting it Altogether

### References
1. [Introduction to Merkle Tree by Geeks for Geeks](https://www.geeksforgeeks.org/introduction-to-merkle-tree/) 
2. [What are Merkle trees? By Alchemy](https://www.alchemy.com/docs/what-are-merkle-trees)
3. [Balanced Binary Tree by Geeks for Geeks](https://www.geeksforgeeks.org/balanced-binary-tree/)
4. [Merkle Trees and Merkle Proofs in Solidity Smart Contracts by Ciara Nightingale](https://www.youtube.com/watch?v=s7C2KjZ9n2U)
5. [Balanced Binary Tree by Progamiz](https://www.programiz.com/dsa/balanced-binary-tree#:~:text=A%20balanced%20binary%20tree%2C%20also,the%20right%20subtree%20is%20balanced)